## Necessary imports

In [1]:
import pandas as pd

from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

## Create query function

The function below takes a SPARQL query string, queries WikiData, and returns the result as a pandas DataFrame (a table).

In [2]:
def query_wikidata(sparql_query):
    """
    Query Wikidata with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to WikiData
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask Wikidata for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

## Run our SPARQL query

In [3]:
sparql_query = """SELECT ?drug ?drugLabel ?gene ?geneLabel ?entrez_id ?disease ?diseaseLabel WHERE {
      ?drug wdt:P129 ?gene_product .   # drug interacts with a gene_product 
      ?gene_product wdt:P702 ?gene .  # gene_product is encoded by a gene
      ?gene wdt:P2293 ?disease .    # gene is genetically associated with a disease 
      ?gene wdt:P351 ?entrez_id .  # get the entrez gene id for the gene 
      # add labels
      SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
      }
    }
    limit 1000
    """

result_table = query_wikidata(sparql_query)

## Look at the results of our SPARQL query

In [4]:
result_table.shape

(595, 17)

In [5]:
result_table.head()

,disease.type,disease.value,diseaseLabel.type,diseaseLabel.value,diseaseLabel.xml:lang,drug.type,drug.value,drugLabel.type,drugLabel.value,drugLabel.xml:lang,entrez_id.type,entrez_id.value,gene.type,gene.value,geneLabel.type,geneLabel.value,geneLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q60235,literal,Caffeine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
1,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q190012,literal,Adenosine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
2,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q407308,literal,Theophylline,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
3,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q729213,literal,Nicardipine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
4,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q905783,literal,Istradefylline,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en


Our results from the SPARQL query has more detail than we care about. We only want to know the names of the drug, disease, and gene columns.

## Extract only the columns we care about

In [6]:
simple_table = result_table[["drugLabel.value", "diseaseLabel.value", "geneLabel.value"]]

In [7]:
simple_table.head()

,drugLabel.value,diseaseLabel.value,geneLabel.value
0,Caffeine,obesity,adenosine A3 receptor
1,Adenosine,obesity,adenosine A3 receptor
2,Theophylline,obesity,adenosine A3 receptor
3,Nicardipine,obesity,adenosine A3 receptor
4,Istradefylline,obesity,adenosine A3 receptor


### Rename the columns of our simple table

Let's delete the "Label.value" portion from the column names.

In [8]:
simple_table = simple_table.rename(columns = lambda col: col.replace("Label.value", ""))

In [9]:
simple_table.head()

,drug,disease,gene
0,Caffeine,obesity,adenosine A3 receptor
1,Adenosine,obesity,adenosine A3 receptor
2,Theophylline,obesity,adenosine A3 receptor
3,Nicardipine,obesity,adenosine A3 receptor
4,Istradefylline,obesity,adenosine A3 receptor


We have just grabbed the three columns we care about and renamed them.

## Count the number of genes per (drug, disease) pair

How many genes link each unique (drug, disease) pair? After counting, order the (drug, disease) pairs in descending order of number of linking genes.

### Make unique (drug, disease) groups and count the number of genes for this group

In [10]:
counts = simple_table.groupby(["drug", "disease"]).size()

In [11]:
counts.head()

drug                                             disease                  
(2S,3S)-2-amino-3-phenylmethoxybutanedioic acid  essential tremor             1
                                                 schizophrenia                1
1,9-Pyrazoloanthrone                             peripheral artery disease    1
2-Aminoethoxydiphenyl borate                     obesity                      1
2-methyl-6-(phenylethynyl)pyridine               Rheumatoid Arthritis         1
dtype: int64

### Turn result into a table

In [12]:
counts = counts.to_frame("gene_count")

In [13]:
counts.head()

gene_count
drug                                            disease                              
(2S,3S)-2-amino-3-phenylmethoxybutanedioic acid essential tremor                    1
                                                schizophrenia                       1
1,9-Pyrazoloanthrone                            peripheral artery disease           1
2-Aminoethoxydiphenyl borate                    obesity                             1
2-methyl-6-(phenylethynyl)pyridine              Rheumatoid Arthritis                1

### Put the drug and disease labels back into columns

In [14]:
counts = counts.reset_index()

In [15]:
counts.head()

,drug,disease,gene_count
0,"(2S,3S)-2-amino-3-phenylmethoxybutanedioic acid",essential tremor,1
1,"(2S,3S)-2-amino-3-phenylmethoxybutanedioic acid",schizophrenia,1
2,"1,9-Pyrazoloanthrone",peripheral artery disease,1
3,2-Aminoethoxydiphenyl borate,obesity,1
4,2-methyl-6-(phenylethynyl)pyridine,Rheumatoid Arthritis,1


### Sort the table in descending order of genes

In [16]:
counts = counts.sort_values("gene_count", ascending = False)

In [17]:
counts.head()

,drug,disease,gene_count
124,Caffeine,obesity,3
437,Quisinostat,obesity,2
32,Adenosine triphosphate,obesity,2
529,Trichostatin A,obesity,2
390,Panobinostat,obesity,2


### Number the results from 0 onwards

In [18]:
counts = counts.reset_index(drop = True)

In [19]:
counts.head()

,drug,disease,gene_count
0,Caffeine,obesity,3
1,Quisinostat,obesity,2
2,Adenosine triphosphate,obesity,2
3,Trichostatin A,obesity,2
4,Panobinostat,obesity,2


---

## Add the genes used to link each (drug, disease) pair

Now add another column containing the actual genes linking each (drug, disease) pair.

### Create a dictionary containing the linking genes for each unique pair

In [20]:
linking_genes = dict()
for (drug, disease), small_table in simple_table.groupby(["drug", "disease"]):
    linking_genes[(drug, disease)] = list(small_table["gene"])

### Example: linking genes for (caffeine, obesity)

In [21]:
linking_genes[("Caffeine", "obesity")]

['adenosine A3 receptor',
 'inositol 1,4,5-trisphosphate receptor, type 1',
 'ryanodine receptor 2 (cardiac)']

### Make a new column containing the linking genes

In [22]:
counts["genes"] = counts[["drug", "disease"]].apply(
    lambda row: linking_genes[(row["drug"], row["disease"])],
    axis = 1
)

In [23]:
counts.head()

,drug,disease,gene_count,genes
0,Caffeine,obesity,3,"[adenosine A3 receptor, inositol 1,4,5-trispho..."
1,Quisinostat,obesity,2,"[histone deacetylase 9, histone deacetylase 7]"
2,Adenosine triphosphate,obesity,2,"[inositol 1,4,5-trisphosphate receptor, type 1..."
3,Trichostatin A,obesity,2,"[histone deacetylase 9, histone deacetylase 7]"
4,Panobinostat,obesity,2,"[histone deacetylase 9, histone deacetylase 7]"


## Save to file

In [24]:
counts.to_csv("drug_disease_count.tsv", sep = '\t', index = False)

---

## Make cytoscape file

We want a file with all the edges for Cytoscape.

### Start with the drug and gene pairs

In [25]:
drug_gene_links = simple_table[["drug", "gene"]]

In [26]:
drug_gene_links.head()

,drug,gene
0,Caffeine,adenosine A3 receptor
1,Adenosine,adenosine A3 receptor
2,Theophylline,adenosine A3 receptor
3,Nicardipine,adenosine A3 receptor
4,Istradefylline,adenosine A3 receptor


### Rename the columns

In [27]:
drug_gene_links = drug_gene_links.rename(columns = {"drug": "source_node", "gene": "target_node"})

In [28]:
drug_gene_links.head()

,source_node,target_node
0,Caffeine,adenosine A3 receptor
1,Adenosine,adenosine A3 receptor
2,Theophylline,adenosine A3 receptor
3,Nicardipine,adenosine A3 receptor
4,Istradefylline,adenosine A3 receptor


### Create a new column specifying the source node type

In [29]:
drug_gene_links["source_type"] = "drug"

In [30]:
drug_gene_links.head()

,source_node,target_node,source_type
0,Caffeine,adenosine A3 receptor,drug
1,Adenosine,adenosine A3 receptor,drug
2,Theophylline,adenosine A3 receptor,drug
3,Nicardipine,adenosine A3 receptor,drug
4,Istradefylline,adenosine A3 receptor,drug


### Create a new column containing the edge type

In [31]:
drug_gene_links["edge_type"] = "interacts_with"

In [32]:
drug_gene_links.head()

,source_node,target_node,source_type,edge_type
0,Caffeine,adenosine A3 receptor,drug,interacts_with
1,Adenosine,adenosine A3 receptor,drug,interacts_with
2,Theophylline,adenosine A3 receptor,drug,interacts_with
3,Nicardipine,adenosine A3 receptor,drug,interacts_with
4,Istradefylline,adenosine A3 receptor,drug,interacts_with


### Create a new column containing the target type

In [33]:
drug_gene_links["target_type"] = "gene"

In [34]:
drug_gene_links.head()

,source_node,target_node,source_type,edge_type,target_type
0,Caffeine,adenosine A3 receptor,drug,interacts_with,gene
1,Adenosine,adenosine A3 receptor,drug,interacts_with,gene
2,Theophylline,adenosine A3 receptor,drug,interacts_with,gene
3,Nicardipine,adenosine A3 receptor,drug,interacts_with,gene
4,Istradefylline,adenosine A3 receptor,drug,interacts_with,gene


## Repeat for disease gene pairs

In [35]:
disease_gene_links = simple_table[["disease", "gene"]]

In [36]:
disease_gene_links.head()

,disease,gene
0,obesity,adenosine A3 receptor
1,obesity,adenosine A3 receptor
2,obesity,adenosine A3 receptor
3,obesity,adenosine A3 receptor
4,obesity,adenosine A3 receptor


## Rename the columns

In [37]:
disease_gene_links = disease_gene_links.rename(columns = {"disease": "source_node", "gene": "target_node"})

In [38]:
disease_gene_links.head()

,source_node,target_node
0,obesity,adenosine A3 receptor
1,obesity,adenosine A3 receptor
2,obesity,adenosine A3 receptor
3,obesity,adenosine A3 receptor
4,obesity,adenosine A3 receptor


### Create the new columns

In [39]:
disease_gene_links["source_type"] = "disease"
disease_gene_links["edge_type"] = "associated_with"
disease_gene_links["target_type"] = "gene"

In [40]:
disease_gene_links.head()

,source_node,target_node,source_type,edge_type,target_type
0,obesity,adenosine A3 receptor,disease,associated_with,gene
1,obesity,adenosine A3 receptor,disease,associated_with,gene
2,obesity,adenosine A3 receptor,disease,associated_with,gene
3,obesity,adenosine A3 receptor,disease,associated_with,gene
4,obesity,adenosine A3 receptor,disease,associated_with,gene


# Join the (disease, gene) and (drug, gene) tables together

### Number of rows of each table

In [41]:
len(drug_gene_links)

595

In [42]:
len(disease_gene_links)

595

### Join two tables together

In [43]:
cytoscape_edges = pd.concat([drug_gene_links, disease_gene_links])

In [44]:
cytoscape_edges.head()

,source_node,target_node,source_type,edge_type,target_type
0,Caffeine,adenosine A3 receptor,drug,interacts_with,gene
1,Adenosine,adenosine A3 receptor,drug,interacts_with,gene
2,Theophylline,adenosine A3 receptor,drug,interacts_with,gene
3,Nicardipine,adenosine A3 receptor,drug,interacts_with,gene
4,Istradefylline,adenosine A3 receptor,drug,interacts_with,gene


In [45]:
len(cytoscape_edges)

1190

Note that the final result has 1190 (= 595 * 2) rows.

### Reorder columns

In [46]:
cytoscape_edges = cytoscape_edges[["source_node", "source_type", "edge_type", "target_node", "target_type"]]

In [47]:
cytoscape_edges.head()

,source_node,source_type,edge_type,target_node,target_type
0,Caffeine,drug,interacts_with,adenosine A3 receptor,gene
1,Adenosine,drug,interacts_with,adenosine A3 receptor,gene
2,Theophylline,drug,interacts_with,adenosine A3 receptor,gene
3,Nicardipine,drug,interacts_with,adenosine A3 receptor,gene
4,Istradefylline,drug,interacts_with,adenosine A3 receptor,gene


## Save file to disk

In [48]:
cytoscape_edges.to_csv("drug_gene_disease_network.txt", sep = '\t', index = False)